# Team members

### Felipe Ramirez (331471)

Bachelor en Microtechnique

### Paul-Antoine Croux (327964)
Bachelor Microtechnics 
Born in Lausanne
More intrested in mechanical design than elecronical design
Genuinely like optimisation and algorythm compared to interface coding
### Seif Labib (326481)

### Nathann Morand (296190)
Electronicien CFC.

Work as a system administrator beside EPFL.

Socially not very aware.

Can code in Python.

# Environment
Our work environment is quite diverse, integrating various tools and platforms. We primarily use PyCharm and VSCode as IDEs, with Git for version control. Some team members prefer GitKraken or GitHub’s web interface. To minimize merging issues, we predefined a class skeleton to structure each team member's work. Function names and return types were also established beforehand to reduce incompatibilities and simplify testing.

As for the robot’s environment, we designed a scenario where the robot acts as an ambulance. Its mission is to reach the site of an accident by navigating roads and avoiding cars (obstacles) along its path. The onboard camera functions as a “GPS,” providing absolute positioning, though it is subject to temporary outages and cannot detect dynamic obstacles. The robot and target locations are marked using two ArUco markers in the form of QR codes, while a checkerboard is used to calibrate the camera and determine the scale in millimeters. The robot is controlled asynchronously via Python.



# Design choice

### Architecture overview (from README.md)
Main loop :
- get the Camera position of the map and robot and the target (car crash, ciff)
- get the map layout (road line) (Camera)
- get the position from the robot encoder
- use kalmann to improve robot position
- update robot position
- use path planning to find best way (A*)
- update robot movement instruction

Fast loop :
- react to obstacle and avoid,
- follow track given by path planning

Pathplanning :
- use the occupancy matrix to compute the fastest route

ComputerVision :
- get the image dimmension + calibrate the camera.
- get the robot + goal position
- get the occupancy map

Robot :
deal with the hardware + realtime
- give estimated position
- allow the update of the estimated position
- get a list of waypoint and follow them.

sensorfusion :
- merge the data from the camera + encoder to improve the accuracy.

### Robot movement + Local obstacle avoindance -> Seif

python class for robot movement.
this is the fast loop.
It run in its own thread and handle to robot movement  + communication and local obstacle avoidance.

The robot movement is based on a few key elements, namely the position of the robot, being an attribute of the robot that can be accessed and modified with a setter and a getter, then the waypoints, also being an attribute than can be accessed and modified with a setter and a getter, which expresses the desired list of waypoints (positions) that we want the robot to follow in order to get to the target (pathplanner.py, the slower thread) and will follow them, using a PID controller for the forward speed, and a simple P controller for the direction of the robot. Both control steps are done separately as when testing I found that combining them in a single step causes a lot of overshoot (in the cases where the angle difference is so little and the distance between the robot and the target (waypoint) is so large, or the other way around if the angle difference is so large while the distance is so little), this overshoot being very slowly treated by the PID controller, I decided to separate both controls and it was so practical.
For the PID controller, the Ki, Kp, and Kd coefficients were experimentally decided, with the constraint of having the whole system behaviour as stable as it can be, but also as fast or as 'real-time' as we can, because at the end of the day the robot simulates an ambulance car.

In the update() function, where everything is computed and the speeds of the motors are set, we set the position of the robot using two key values, the kalman filter value from the slower loop, as well the estimated position using the wheel encoder of the robot.

For the local avoidance and the speed control of the robot, we had an additional safety variable called obstacle_detec, just in case the robot is detecting something through its IR sensors but still goes on with the for loop and goes to modify its direction and forward speed, this was just a safety measure that worked while testing so I decided to leave it as it is. 
When it comes to the local avoidance algorithm, the algorithm used was basic and was based upon the IR sensor values.
I only used the front sensors, but within these 5 sensor values, we had an internal hierarchy, meaning that not every sensor has the same importance, this hierarchy was simply implemented using different threshold for the different sensors.
This hierarchy was based on the fact that the robot can't crash into an obstacle in front of it, but can navigate normally when an obstacle is next to it, even if close. So the highest sensors in our robot were the front center left and front center right, to which we implemented a change of directions, and then a simple forward speed then it rechecks if the obstacle is avoided so it re-corrects its direction and follows the planned path, or if not keep avoiding the obstacle and moving forward and then rechecking whether the obstacle was avoided or not and repeat the loop until the obstacle is completely avoided.
This hierarchy was put into place because the case where the obstacle is so thin it can only be detected by one of the sensors, namely the front center one, without the front center right or left sensors was so extreme and rare. That's why the priority was given to the front center right and left ones, as they provide more info than the front center one, a last case where the robot was completely "surrounded" by obstacles was implemented just in case, for it to be a complete local avoidance algorithm.
For the direction change, the angle change, or rather the direction change was around 1 radian, so around 57 degrees, which at first was a random choice, but then proved to be a good one, as it is not so fast/brusque that it produces overshoot neither is it so small so that we would have to stay in the loop for so long.
While for the simple forward speed, it was the same idea, a random not so high speed was set and then found reasonable seeing the behaviour of the robot, although it could've been set higher (we still have an acceptable marge of error).

Finally, for the testing, the main idea was to test the PID controller, as well as the obstacle avoidance algorithm, so the testing was just a simple square that the robot has to form, where I used different obstacles as my hand, water bottles.


also make the thymio blink in blue like an ambulance.

```
class RobotMovement:
    def __init__(self):
        # Initialize motors, sensors, and encoders
        self.waypoints = [np.array([0, 0, 0])] # list of np.array
        self.position = np.array([0, 0, 0])
        self.speed = np.array([0, 0, 0])

    def set_waypoints(self, waypoints): # Beware, this class is inside a thread you need to use a mutex !!!
        # same format as get_waypoints
        # update the global list of waypoint to follow

    def get_waypoints(self, waypoints): # Beware, this class is inside a thread you need to use a mutex !!!
        # return the global list of waypoint to follow minus the one that where reached aldready
        waypoints = [
            np.array([100, 150, np.pi / 6]),  # x=100mm, y=150mm, direction=30° (π/6 radians) clockwise rotation from x axis
            np.array([200, 250, np.pi / 3]),  # x=200mm, y=250mm, direction=60° (π/3 radians)
            np.array([300, 350, np.pi / 2])   # x=300mm, y=350mm, direction=90° (π/2 radians)
        ]
        return waypoints

    def get_position(self):# Beware, this class is inside a thread you need to use a mutex !!!
        # Return estimated position from encoders
        # same format as for set_position

    def set_position(self, kalman_position):# Beware, this class is inside a thread you need to use a mutex !!!
        # Update the robot’s position based on Kalman filter results
        return np.array([100, 150, np.pi / 6]),  # x=100mm, y=150mm, direction=30° (π/6 radians)

    def get_speed(self): # there is no need to set the speed from kalman, the motor know better
        # Return the speed+angular speed from encoders
        # Beware, this class is inside a thread you need to use a mutex !!!
        return np.array([10, 15, np.pi / 60]),  # x=10mm/s, y=15mm/s, direction=3°/s (π/60 radians/second)

    def update(self):# this function is called every 10 ms for you to do what you have to
                     # (avoid obstacle, go to the nearest waypoint)
        return None
```
### Computer vision and global positionning + occupancy grid -> Felipe
This class sets up a computer vision system using OpenCV, with the main goal of generating an occupancy grid and position of our robot and goal. This grid represent the environment as occupied and free area, based on images captured from a camera or loaded from a file. The grid will then be used by other modules/class in the project for tasks like navigation or pathplanning.

The VisionSystem class offers a way to capture images, allowing the use of either a real-time camera or image files, which is especially useful for testing. It uses ArUco markers to detect two key elements: the robot and the goal. Using these marker, the class can locate these objects in the environment, calculating their positions in millimeters and their orientations.

Camera calibration is included to improve the accuracy of measurements. It uses a checkerboard pattern to adjust the camera parameters and calculate the size of a pixel in millimeter. This ensures that image coordinates can be converted into real-world distances.

Finally, the main method, which generates the occupancy grid, processes the image to identify fixed obstacles in the environment. It converts the image to grayscale, applies a filter to reduce noise, and binarizes the image to create a clear map of occupied and free areas. The resulting grid will then be used by other project modules.

In summary, this class provides an occupancy grid (NxM matrix) and return the position and orientation of the robot used for path planning.

To provide more clarity, let me explain some key aspects of our code:

We chose to use ArUco markers because they provide both position and orientation and are relatively fast for a computer to process.
We opted for calibration with a large checkerboard instead of ArUco markers because ArUco markers are small, which limited the precision of the pixel-to-millimeter conversion. 

So we use an image of a 8x8 checkerboard to estimate the camera parameters and calculate the pixel size in millimeters. The image is captured, converted to grayscale, and the checkerboard corners are detected and refined for better accuracy. Using the detected corners and the real dimensions of the checkerboard, the camera matrix and distortion coefficients are calculated. The pixel size is then estimated by measuring the distances between the checkerboard corners in pixels and comparing them to their real-world dimensions. If successful, the calculated parameters are displayed; otherwise, a message indicates the calibration failed.

For the detection of the ArUco markers we use the function cv2.aruco.detectMarkers() from OpenCV. This function is a encapsulation several processes into a single call. We found a good explanation on internet about what process is used. It's divide in two main steps : https://docs.opencv.org/4.x/d5/dae/tutorial_aruco_detection.html
```
- Detection of marker candidates. In this step the image is analyzed in order to find square shapes that are candidates to be markers. It begins with an adaptive thresholding to segment the markers, then contours are extracted from the thresholded image and those that are not convex or do not approximate to a square shape are discarded. Some extra filtering is also applied (removing contours that are too small or too big, removing contours too close to each other, etc).

- After the candidate detection, it is necessary to determine if they are actually markers by analyzing their inner codification. This step starts by extracting the marker bits of each marker. To do so, a perspective transformation is first applied to obtain the marker in its canonical form. Then, the canonical image is thresholded using Otsu to separate white and black bits. The image is divided into different cells according to the marker size and the border size. Then the number of black or white pixels in each cell is counted to determine if it is a white or a black bit. Finally, the bits are analyzed to determine if the marker belongs to the specific dictionary. Error correction techniques are employed when necessary.
```

For each detected marker, we calculated its center to simplify the next steps, as this provides the precise position of the objects (Robot and goal). Next, we verified whether the desired ID is present in the previously created matrix to ensure the object was detected. Finally, we constructed a vector, vector_x, to determine the angle between the marker and the horizontal axis. Like this we return a np.array() with the position x-y and the orientation of our object. 


### Path Planning -> Paul-Antoine
This class serves to compute the path for the robot to take with the known map of the environment. It is made of different functions to compute the pathway and one function that is called from the outside (get_waypoints()) that returns an array of waypoints that the Thymio has to meticulously follow to reach the goal.  
The functions are the following and will be explained in detail.

---

- init()

- is_valid_point()

- is_line_colliding()

- scale_and_flip_waypoints()

- compute_angles_for_waypoints()

- compute_enhance_grid()

- get_waypoints()

- a_star()

---

The algorithm and the functions have been rigorously described in pseudocode to ChatGPT so it could offer a proper, well-described Python code. Only a_star() has not been detailedly described to ChatGPT as it is a well-known algorithm. It has been asked to ChatGPT to generate the a* function from scratch. We asked him for an a* model because it seemed to be the best compromise to avoid long computing and assuring the return of a working short path if it exists. The only things imposed to ChatGPT to generate the a* function were the parameters and the use of is_line_colliding() to validate that the current vector to be evaluated is not colliding with a wall of the map.

We initially designed the algorithm in such a way that it would be easy to locate misbehaviors of the code generated by ChatGPT. Well-described and fragmented pseudocode allowed us to have working code rapidly. It was only after running-in and testing that we've been exposed to the poor performance of computation time of the algorithm.

The initial algorithm worked as follows:  
It first went through the occupancy_grid (which is the Boolean map with value TRUE for a wall and FALSE for the road), storing in an array all the coordinates with value FALSE. Then it would randomly pick a certain percentage of the coordinates to place, on the map, randomized waypoints to be evaluated. One after the other, each waypoint would be evaluated before being officially placed. Before officially placing the point, each point is tested to see if it's in the range of a wall or another already placed point, in which case, the point does not become a placed point. This was done with the function is_valid_point(). It allows us to place waypoints not too close to each other and not too close to a wall so the Thymio doesn't hit a wall trying to reach the point. Then this array of placed points is given to the a_star() function for it to evaluate all the nodes. Each vector of two nodes is processed in the a_star() by going through is_line_colliding(), which evaluates if the straight line between two nodes goes through a wall. is_line_colliding() used to take the slope of the two nodes and then increment x and evaluate y as the slope multiplied by the increment. For each increment we would look if the occupancy grid is FALSE (meaning we have a free way) and also check for all the coordinates within a radius to check if the hitbox of the Thymio wouldn't hit a wall while getting to the next node. From there we let the a_star() compute all nodes and return the most optimized set of waypoints checking all the hitbox requirements along the path.

After testing, we've noticed the computation time to be between 1.5 and 2 seconds long. And as the function get_waypoints() was called in the slow loop of the main, we've realized the issue that this computation time represents. From this, we concluded that there were two main flaws in the design of the algorithm. The first being the way we check is_line_colliding(). As the a_star() computing increases exponentially with the number of nodes to be evaluated, the calling of is_line_colliding() for every pair of nodes checked, aggravates the computing time. When is_line_colliding() checks all the coordinates along a line and also checks the coordinates within a radius of each coordinate, we easily see that some coordinates are going to be checked over and over, adding redundancy in the checking of the occupancy grid that can be avoided to enhance the computation time. The second flaw is also in is_line_colliding() when it increments x with the corresponding y (computed with the slope and the value of x); it actually jumps over walls if the slope is higher than 1. If dx is just about a few, we are going to evaluate only dx points where the actual magnitude can be way bigger if dy exceeds dx.

From there we made adjustments, but not with ChatGPT. It already did its job furnishing a working algorithm that we could simply remaster to optimize the computing time. The first change was after noticing that we could set a new Boolean map with the hitbox of the Thymio included. By computing, only once, the enhanced grid with the function compute_enhance_grid(), we could take out the part in is_line_colliding() that is computing a radius hitbox for all coordinates of a line. As the enhanced grid would already take into account all positions that the center of Thymio can be at without colliding with a wall. In is_valid_point() we can also take out the radius computation, as if we take the enhanced grid, we already consider Thymio's hitbox.

The second flaw that we fixed was the increment of x with y as a function of the slope and the increment in is_line_colliding(). To remedy this problem, we simply put the increment on y if the slope is greater than 1. This allows us to ensure proper incrementation and to test the maximum number of cells within the line to be evaluated. To reduce the computation time by a small factor, we also decided to evaluate only one cell every few increments along the line. The current cell evaluated is only checked once every 5 increments, which lowers the computing time of is_line_colliding() by a factor of 5. We only have to ensure that walls are never thinner than 5 cells, and that if the line encounters a corner, the error of jumping over the corner does not mean the Thymio will hit the corner (which is considered in the hitbox of Thymio for a 5-cell jump).

---


Here is an example of all the placed random waypoints that are going to be evaluated by the a_star():

![legende](testData/MapRapport4.jpg)

---

Here, we show how all the possible waypoints are placed on the enhanced grid that considers Thymio's hitbox:

![legende](testData/MapRapport3.jpg)

---

After the a_star() computation, the function get_waypoints(), called from the outside, returns the set of waypoints that the Thymio has to follow:

![legende](testData/MapRapport2.jpg)

---

In the following image, we can see the returned waypoints on the enhanced grid, to visualize that each line between nodes is indeed in values FALSE of the enhanced grid, meaning that it won't hit any walls if following the waypoints with stationary rotation on each waypoint to get to the next.

![legende](testData/MapRapport1.jpg)

---

In the end, we've reached a one-second computation for the first time of the loop, but if the get_waypoints() is called again to adjust its path, it will take only 0.1 second to compute a new set of waypoints. We've also included scale_and_flip_waypoints() to adjust the coordinates so we can show the waypoints on the image with the origin in the bottom-left corner. The function compute_angles_for_waypoints() has been created so we can return the absolute angle of each waypoint. This is just an option that can be used as additional information about the waypoints if needed for more precise control of the Thymio as we had originally disigned the control of thymio.

---
You can simply run the pathplanner.py file to see the simulation of the waypoints beeing generated. By running the file you will see the path for a simpler road that has been taken with the camera, in which the goal et the robot are positioned from a QR-code.



### sensor fusion and test script -> Nathann
I tried at first to implement a kalman filter that worked relatively well in "static" condition (linear acceleration or constant speed)
![Kalman Early Test](testData/kalmanEarlyTest.jpg)
but since the robot is changing course from time to time I did not like this solution.
instead I implemented a particule filter that was more robust to the constantly changing course.
![Particule Filter](testData/particuleFilter.jpg)
The particle filter class implement the filter itself (along each DOF of the robot so X position, Y position and theta) and offer the following function :

init: Create the particle filter with a specified number of particles and the max range (we assume that the robot will not travel outside an 10x10 meter arena nor will do more than one full turn on itself before rewinding)

Predict: Move particles by a distance obtained by mutliplying the current speed by the elapsed time and adding some noise. (the noise amplitude is determined empirically to be of the same magnitude of the noise from the sensor)

Update: this function compute the weight (higher mean more likely to reflect the true position of the robot) of each particle compared to the value returned from the sensor. (when both the camera + robot odometry are avaible, do the average between the two)

Resample: This function act as a lotery where a higher weight mean more chance of winning. it take all the "winning" particle (some particle win multiple time) and replace the population of N particle with N new particle made of the winner by the number of win of each winner. this is a zero sum game so that the particle number stay constant.

Estimate: compute the mean position of the particles to find a close approximation of the true position of the robot.

The sensor fusion class keep track of the 3 instance of the particle filter (for each DOF) and check that the value are available for each sensor (if yes, compute the average of the two sensor, if not only use the data from one sensor.

I also made some test script to help the others to verify that the code was working. They take the form of a
"if __name__ == "__main__":" in each file to test the file individually with the help of a visualisation tool.

The Visualisation library offer function function to :

update_plot : This take into argument the posisiton of the robot from the robot and the camera + the goal position and the estimated position. It also take the list of waypoints to reach the target and the occupancy grid and the raw image from openCV and make a nice chat using matplotlib :
![visualisation](testData/visualisation.jpg)

plot_robot_grid : this is similar to update_plot but more crude as it does not update the plot dynamically and is used for static test of the pathfinding algoritm and the computer vision test.

Finally I wrote guy.py as the matplotlib dynamic plot was not working reliably on every OS.
It bundle the whole programm and offer user friendly button to enable the simulation of some module and tune some parameters like the camera ID.
![visualisation](testData/gui.png)

# Demo
For a full demo you can run gui.py by following the bash command shown after.

Each module can be individually run and tested. sensorfusion generate a plot that show how it behave to a simulated input from two noisy sensor. pathplanner load an occupancy grid and show a visualisation of the path found. robot movement was supposed to follow a set of waypoint and
Vision system show the occupancygrid from a picture and the goal + robot found using their marker.

to setup your environement clone and cd in this directory and run the following command (tested on ubuntu 20) (BEWARE, REQUIRE PYTHON >= 3.11)

In [ ]:
python3.12 -m venv .venv # setup a virtual environement to prevent library issue
source .venv/bin/activate  # or .venv\Scripts\activate on Windows
pip install -r requirements.txt # install our required library
python3.12 gui.py # python3 test.py

# Conclusion

Even tho our project is not fully functional some module work well and have been toroughly tested. We are especially happy with the approach of writing the class and define ahead how the module will interact to try to catch has many bug ahead as possible.